# Strings

***DISCLAIMER: Este notebook foi escrito com base no que li [neste](https://docs.julialang.org/en/v1/manual/strings/) capítulo do manual***


Em outras linguagens é muito comum utilizarmos apenas os caracteres mais básicos, como letras do alfabeto, números e alguns sinais de pontuação, e isso encaixa perfeitamente com o padrão ASCII.

Porém já vimos que Julia suporta Unicode e LaTeX na criação de nomes de variáveis, logo podemos supor, e bem, que ela dará suporte a Unicode em strings.

Basicamente Julia tem $2$ grandes tipos: `AbstractString` e o `AbstractChar` - e daí surge outros subtipos como: `String` e `Char` - onde estes últimos $2$ são apenas para strings e caracteres ASCII. 

Portanto, devemos sempre definir os parâmetros das nossas funções que vão receber strings ou caracteres como um dos grandes tipos.

## Caracteres

Caracter é quando temos apenas uma letra, número, simbolo ou sinal, ou seja, não é um """texto""".

Definimos um caracter fazendo o uso de plicas `''`.

In [74]:
# Atribuir o caracter A à nossa variável
caracter = 'A'

'A': ASCII/Unicode U+0041 (category Lu: Letter, uppercase)

In [75]:
typeof(caracter)

Char

Podemos converter caracteres em decimais (o decimal corresponde ao seu **code point**, isto é, a sua posição na tabela ASCII e/ou à sua representação como ponto Unicode).

In [76]:
decimal = Int(caracter)

65

E podemos reverter isso usando o subtipo `Char`.

In [77]:
Char(decimal)

'A': ASCII/Unicode U+0041 (category Lu: Letter, uppercase)

Para verificar se um decimal ou hexadecimal é válido como ASCII/Unicode, utilizamos a função `isvalid`.

In [78]:
isvalid(Char, 0x110000)

false

In [79]:
isvalid(Char, 0x2200)

true

Para escrevemos um caracter Unicode podemos fazer uso de `\u` ou `\U` 

In [80]:
'\u2200'

'∀': Unicode U+2200 (category Sm: Symbol, math)

In [81]:
# Sigma maiúsculo
'\u03a3'

'Σ': Unicode U+03A3 (category Lu: Letter, uppercase)

Julia sabe se pode printar certos caracteres ou se precisa de recorrer à sua representação Unicode. Isto porque a linguagem tem acesso à linguagem e á região definidas no sistema operativo.

## Strings

Para criar strings utilizamos as aspas `""`. E caso queiramos utilizar as aspas dentro da strings temos que fazer um escape com o backslash`\`.

In [82]:
println("Sou uma string")

Sou uma string


In [83]:
println("Sou uma string com \"aspas\"")

Sou uma string com "aspas"


Para extrair caracteres de uma string, podemos tratá-la como um array e acedemos através dos índices.

Caso queiramos o primeiro ou último caracter utilizamos as palavras reservadas `begin` e `end` como índices.

Porém essas palavras são apenas um shorthand, um "atalho" para as funções `firstindex(str)` e `lastindex(str)`.

In [84]:
str = "Sou um teste"

str[begin]

'S': ASCII/Unicode U+0053 (category Lu: Letter, uppercase)

In [85]:
str[end]

'e': ASCII/Unicode U+0065 (category Ll: Letter, lowercase)

In [86]:
str[1]

'S': ASCII/Unicode U+0053 (category Lu: Letter, uppercase)

***Já devem ter reparado que Julia tem algo igual ao MATLAB ... A indexação.***

***Portanto os índices não começam em 0 mas sim em 1. Estes pormenores e outros que já apareceram ou vão aparecer mais à frente, são consequências de Julia ser uma linguagem voltada para a área científica (matemática).***

Assim como no Python, podemos extrair substrings utilizando: `[indice_inicio:indice_fim]`

In [87]:
str[1:3]

"Sou"

***MAS*** o senhor Python habitua-nos muito mal, pois se definirmos uma string com `''` é o mesmo que definir com `""` e tudo é considerado string.

Pois é ... aqui em Julia isto `str[6]` é diferente disto `str[6:6]`, porque `str[6]` está-nos a retornar o caracter no índice 6, porém `str[6:6]` está a retornar uma substring que começa no índice 6 e termina no índice 6. 

Resumo: um é do tipo `Char` e o outro é do tipo `String`.

In [88]:
typeof( str[6] )

Char

In [89]:
typeof( str[6:6] )

String

Também se pode obter substrings através do tipo `SubString`, fazendo com que tenhamos um objeto desse tipo e não do tipo `String`.

In [90]:
sub = SubString(str, 1, 4)

typeof(sub)

SubString{String}

## Unicode

Quando definimos alguma string utilizando o `\u`, ela será codificada em UTF-8 o que faz com que cada caracter possa ocupar um número de bytes superior a 1 (normalmente do code point 128 para cima, os caracteres ocupam mais que 1 byte).


Isto fará com que quando tentamos aceder a um caracter de índice $n$, nem sempre será válido, pois podemos apanhar algum byte referente ao caracter do índice anterior.

In [91]:
s = "\u2200 x \u2203 y"

length(s)

7

In [92]:
# O primeiro índice representará o primeiro caracter
s[1]

'∀': Unicode U+2200 (category Sm: Symbol, math)

In [93]:
# Como o caracter anterior é um Unicode com code point > 128, o índice 2 ainda pertence a ele
# por isso ele é um índice inválido para ser acedido
s[2]

LoadError: StringIndexError: invalid index [2], valid nearby indices [1]=>'∀', [4]=>' '

In [94]:
s[3]

LoadError: StringIndexError: invalid index [3], valid nearby indices [1]=>'∀', [4]=>' '

Bem o símbolo ∀ ocupou $3$ bytes da nossa string, mas se repararmos Julia avisa, no erro, qual o próximo caracter válido para printar: `StringIndexError: invalid index [3], valid nearby indices [1]=>'∀', [4]=>' '
` - Neste caso o índice 4 é o próximo índice válido.

In [95]:
s[4]

' ': ASCII/Unicode U+0020 (category Zs: Separator, space)

Para evitarmos ter que verificar todos os índices e olhar para a mensagem de erro, podemos utilizar a função `nextind(str, indice)`, onde o `indice` é o índice a partir do qual queremos verificar o próximo índice válido. Por exemplo, se eu passa-se a nossa string `s` e o índice $1$, a função retornaria $4$.


Também há a `prevind(string, indice)` caso queiramos ver de trás para a frente, ou seja, o índice válido anterior ao que passámos como argumento.

Podemos passar um argumento opcional, sendo esse o valor válido que queremos. Por exemplo, se eu quiser o índice válido a seguir ao primeiro índice, faço: `nextind(s, firstindex(s))` - MAS, se eu quiser não o valor a seguir, mas o valor ... a seguir ao a seguir, ou seja, o segundo valor válido depois do primeiro índice, faço: `nextind(s, firstindex(s), 2)`.

In [96]:
# índice do valor válido a seguir ao primeiro indice
nextind(s, firstindex(s))

4

In [97]:
# índice do segundo valor válido a seguir ao primeiro indice
nextind(s, firstindex(s), 2)

5

***Nota:*** &#8595;

<hr>

`length(string) <= lastindex(s)` - pois o length vai contar como se cada caracter ocupasse apenas 1 byte, apenas 1 índice.

<hr>

Por fim, se quisermos iterar por todos os possíveis índices em uma string, utilizamos a função `eachindex(string)` que já fará de forma otimizada.


***Nota:*** &#8595;

<hr>

Ela retornará um objeto do tipo `EachStringIndex` que não é a melhor forma de visualizar os índices. Para isso precisamos converter o objeto em um array/vector de valores (que no caso são índices) utilizando uma outra função `collect(object)`. 

<hr>

In [98]:
collect( eachindex(s) )

7-element Vector{Int64}:
  1
  4
  5
  6
  7
 10
 11

Podemos ver a code unit crua ( representação do(s) byte(s) ) de um dado caracter utilizando o `codeunit(string, indice)`. 

Podemos mostrar todos os code units usando `codeunits(string)`. 

In [99]:
# Code unit do valor no índice 2
codeunit(s, 2)

0x88

In [100]:
codeunits(s)

11-element Base.CodeUnits{UInt8, String}:
 0xe2
 0x88
 0x80
 0x20
 0x78
 0x20
 0xe2
 0x88
 0x83
 0x20
 0x79

Julia aceita qualquer sequência de unit codes, mesmo que contenham UTF-8 codes inválidos. Isso porque ela faz uma verificação da seguinte forma:
* Divide a sequência em sub sequências de 8-bit code units
* Verifica cada uma dessas sub sequências (cada uma delas deve ou não referir-se a um caracter)
* Essas verificação é feita comparando com alguns padrões
* Caso não bata com os padrões Julia dirá se é uma caracter inválido por má formação ou por ter uma sub sequência de code units muito elevado, por exemplo.

In [101]:
# Exemplo mostrado no manual de uma sequência de code units
s = "\xc0\xa0\xe2\x88\xe2|"

"\xc0\xa0\xe2\x88\xe2|"

In [102]:
# Iteramos por todas as sub sequências de 8-bit code units (caracteres)
foreach(display, s)

'\xc0\xa0': [overlong] ASCII/Unicode U+0020 (category Zs: Separator, space)

'\xe2\x88': Malformed UTF-8 (category Ma: Malformed, bad data)

'\xe2': Malformed UTF-8 (category Ma: Malformed, bad data)

'|': ASCII/Unicode U+007C (category Sm: Symbol, math)

Isto não é algo que eu domine muito, mas baseado na documentação vimos que:
* `'\xc0\xa0'` são code units muito altos para representar o caracter espaço ' '
* `'\xe2\x88'` apesar de serem válidos, o próximo code unit (`'\xe2'`) torna esta sub sequência inválida
* `'\xe2'` este além de fazer a sub sequência anterior inválida, também é inválido, pois o caracter posterior `|` não é uma continuação válida.
* `'|'` é um caracter normal


***Nota: O `foreach` é uma forma de loop onde, basicamente, iteramos sobre um objeto e passamos ele para uma função. O `display` é uma função que permite printar o valor de um objeto e ainda obter algumas informações acerca dele.***

Novamente, para não termos de nos basear em erros para verificar o que cada sub sequência tem ou não errado, podemos usar a função `isvalid.(array/vector)` com a nossa `String`.

***Nota*** &#8595; 

<hr>

O `.` depois de `isvalid` significa broadcasting, ou seja, queremos verificar se cada elemento no array é válido(1) ou não(0).

<hr>

In [103]:
isvalid.(collect(s))

4-element BitVector:
 0
 0
 0
 1

Um método útil na nossa jornada é o `transcode(Type, string)`, ele permite-nos converter as nossa strings para outros encoders.

## Concatenação

Começo já por avisar que concatenar strings que possuem sequências de code units inválidas podem gerar outros caracteres.

Mas para mim a parte mais interessante é o operador utilizado para concatenar strings. Em Python usamos o `+` para concatenar strings, porém em Julia usamos o `*`.

Mas porquê ?? Na realidade, faz bastante sentido matematicamente! Quando nós adicionamos 2 matrizes de tamanhos iguais, nós definimos que essa operação é comutativa, porque: $A + B == B + A$- mas, e se em vez de números fossem strings?

$"Jupyter" + "Notebook" \neq "Notebook" + "Jupyter"$ - logo concatenar strings adicionando não faz sentido, MAS utilizando o operador de multiplicação `*` FAZ! Pois a multiplicação de 2 matrizes não é algo comutativo: $A \cdot B \neq B \cdot A$.

Lembrem-se que Julia é uma linguagem com escopo científico, então sempre haverá relações com matemática.

In [104]:
# Uma simples concatenação
"Jupyter " * "Notebook"

"Jupyter Notebook"

## Interpolação

Chegámos ao ponto onde Julia é parecido com Bash. Caso queiramos colocar, por exemplo, uma variável no meio de uma string, em vez de concatenarmos podemos interpolar utilizando `$`:

In [105]:
nome = "User"

"Olá, $nome"

"Olá, User"

Pode-se utilizar para cálculos:

In [106]:
"3 * 3 = $(3*3)"

"3 * 3 = 9"

E no caso de queremos usar o caracter `$`, temos de fazer escape dele com `\`

In [107]:
println("Encontrei 1\$ ao sair de casa")

Encontrei 1$ ao sair de casa


# Strings com aspas triplas

Para escrever textos grandes e com bastante indentação, utilizar as aspas duplas `" "` poderá não ser algo muito prático.

Caso usemos `""" """` para criar a string, podemos colocar espaços, tabulações, quebras de linha etc sem fazer uso de `\n`, `\t`.


Esta forma de definir strings é idêntica ao `f""` do Python.

In [108]:
str = """
Eu sou        um texto
    indentado
"""

println(str)

Eu sou        um texto
    indentado



## Operações básicas

Podemos comparar strings lexicograficamente, ou seja, podemos verificar se uma string é menor que outra com base na posição dos caracteres no alfabeto.

In [109]:
"Sou inferior" < "Sou superior"

true

Podemos encontrar um dado caracter em uma string utilizando as funções: `findfirst` e `findlast` - em que a primeira encontra a primeira ocorrência e a segunda encontra a última ocorrência.


***Nota:*** &#8595;

<hr>

No `isequal` é importante passar um caracter e não uma string, já que a função find vai procurar em cada `Char` presente na string. Se passarmos `"a"` não vai retornar nada, pois é do tipo `String` e não `Char`.

<hr>

In [110]:
findfirst(isequal('a'), "amanhã")

1

In [111]:
findlast(isequal('a'), "amanhã")

3

Podemos encontrar a próxima ou a ocorrência anterior de um dado caracter com as funções: `findnext` e `findprev` - passando um índice que será o offset para procurar, ou seja, começamos a procurar a partir desse índice e se ele for uma das ocorrências, então o próprio índice será retornado.

In [112]:
# Primeira ocorrência a partir do índice 1
findnext(isequal('a'), "amanha", 1)

1

In [113]:
# Primeira ocorrência a partir do índice 2
findnext(isequal('a'), "amanha", 2)

3

In [114]:
# Primeira ocorrência a partir no índice 4 e a contar de trás para frente (inverso)
findprev(isequal('a'), "amanha", 4)

3

A função `occursin` verifica se um `Char` ou uma `String` está dentro da string que passamos como segundo argumento.

In [115]:
occursin("ul", "Julia")

true

Por fim temos mais 3 funções importantes:
* `repeat(str::Str, N::Int)`: Função que cria uma string com a repetição de $N$ vezes de `str`.

* `join([IO::io, ] strs, delim::String, last::String)`: Função que recebe um array de strings, concatena utilizando o `delim` como delimitador entre as strings e `last` como o último delimitador (delimita a penúltima e a última string).

* `length(str::String, i::Int, j::Int)`: Função que retorna o número de caracteres de uma string e caso passemos o $i$ e o $j$, ela apenas retorna o número de caracteres nesse intervalo de índices.

In [116]:
repeat("a", 10)

"aaaaaaaaaa"

In [117]:
join(["Code", "Eat", "Code again", "Repeat"], ", ", " and ")

"Code, Eat, Code again and Repeat"

In [118]:
length("Julia-is-awesome")

16

In [119]:
# Número de caracteres na substring "Julia"
length("Julia-is-awesome", 1, 5)

5

## Strings que fogem ao padrão

Subtítulo bastante interessante, não? Bem, o que eu quero dizer é que até agora manipulámos strings normais.

Porém há formas mais dinâmicas de manipular strings, por exemplo, com o bom e velho *Regex*. 

### Regular Expressions (A.K.A Regex)

As expressões regulares é uma forma de encontrar padrões em strings e é algo bem difícil de entender (eu próprio tenho que entrar [neste site](https://regexr.com/) sempre que quero construir uma regex).


O ponto positivo da complexidade de criar regexs é que a string é passada para uma [State Machine](https://pt.wikipedia.org/wiki/M%C3%A1quina_de_estados_finita) que irá procurar pelos padrões de forma eficiente.

Para criar expressões regulares usamos: `r""`

In [120]:
reg = r"[0-9]"

typeof(reg)

Regex

Podemos verificar se uma string contém um dado padrão com o `occursin()`

In [121]:
# Verificar se existe dígitos na nossa string
occursin(r"[0-9]", "abc123def")

true

Mas a melhor forma é utilizar a função `match(reg::Regex, str::AbstractString)` que irá retornar `nothing` se não encontrar o padrão dentro da string.

Vamos fazer um exemplo rápido: Em Portugal os números de telemóvel (celular em pt-br) contêm 9 dígitos e sempre começam com o algarismo $9$.

Vamos verificar se um número de telemóvel é português com regex.

In [122]:
reg = r"[9]{1}[0-9]{8}"

# Retorna um objeto RegexMatch com a substring onde o padrão está ou nothing
m = match(reg, "912345678")

if m === nothing
    println("NÃO é um número português")
else
    println("É um número português\n")
    
    # Substring que corresponde ao padrão
    println("Substring: ", m.match)
    
    # Índice onde começa o padrão encontrado
    println(m.offset)
end

É um número português

Substring: 912345678
1


Ainda há o:
* `m.captures`: Mostra as diferentes substrings onde se verifica um padrão. Retorna um `Vector`.
* `m.offsets`: Mostra os índices onde começam a diferentes substrings onde se verifica uma padrão. Retorna um `Vector`.

Podemos adicionar um nome a um grupo de padrões e verificar apenas a substring onde se encontra esse grupo. Imaginemos que queremos obter sempre os últimos 3 dígitos do número de telemóvel. Vamos começar por criar a Regex. 

In [123]:
reg = r"[9]{1}[0-9]{5}(?<last>[0-9]{3})"

m = match(reg, "912345678")

m

RegexMatch("912345678", last="678")

Podemos aceder ao nosso grupo `last` utilizando `m[:last]`

In [124]:
m[:last]

"678"

Podemos substituir um padrão por outra coisa qualquer utilizando a função `replace(str::String, reg::Pair`, onde utilizamos `s""` como forma de indicar a expressão que susbtituirá algum padrão na regex.

Podemos referir-nos aos grupos capturado com `\n` - em $n$ que é um inteiro - ou então pelo nome - `\g<nome_do_grupo>`.

Vou utilizar um exemplo do manual.

In [125]:
replace("first second", r"(\w+) (?<agroup>\w+)" => s"\g<agroup> \g<1>")

"second first"

A regex de cima procura 2 padrões separados por um espaço:
* O primeiro é só um conjunto de caracteres alfanuméricos e underscores
* O segundo é o mesmo que o primeiro, porém atribuimos um nome ao grupo


Quando uma dada substring corresponder à regex ela será substituída com base no padrão definido em `s""`, no caso essa padrão, passa o grupo com nome para primeiro, e o primeiro grupo (identificado pelo número da sua posição, `\1`) passa para segundo.


***Nota:*** &#8595;

<hr>

Vimos que a função `replace` recebe um argumento do tipo `Pair`. Pair é algo que iremos ver em dicionários, basicamente é um forma de criar uma par `key:value`, só que em vez de `:` usamos um `=>`.

<hr>

Algo a ter em conta, é que podemos referir-nos a grupos através da sua posição sem ser com o `\n`. Para isso usamos o `\g<n>`, e caso $n = 0$, então iremos mostrar a substring, correspondente ao padrão, completa.

<hr>

Podemos mudar a forma de como o parser opera nas regexs, colocando algumas *flags* no fim da regex.

Flags:
* i: Ativa o Case-insensitive, ou seja, [a-z] é o mesmo que [A-Z].

* m: `^` deixa de ser o caracter que identifica o ínicio da string e passa a ser o caracter que identifica o ínicio de uma linha. O mesmo para o `$` que em vez de identificar o fim de uma string, identifica o fim de uma linha.

* s: Altera o funcionamento do `"."` que por padrão pega qualquer caracter exceto `"\n"`, porém com esta flag, ele inclui o `"\n"`.

* x: Ignora espaços e trata o caracter `"#"` como um metacaracter, o que faz com que ele designe um comentário (como se fosse código normal).

In [126]:
# case-insensitive e ignora os espaços aplicados na regex
reg = r"J u li a"ix

r"J u li a"ix

In [127]:
match(reg, "julia")

RegexMatch("julia")

Por fim, lembro que `r""` não permite interpolação nem escape de caracteres (exceto o `\"`).

Portanto para termos alguma liberdade, podemos usar o *constructor* `Regex()` quer permite-nos criar regexs mais livremente e com as manipulações que fazemos em strings comuns, como a interpolação.

In [128]:
first_digit = 9
end_digits = 3

reg = Regex("[$first_digit]{1}[0-9]{5}(?<last>[0-9]{$end_digits})")

r"[9]{1}[0-9]{5}(?<last>[0-9]{3})"

### Byte Array

Neste notebook, na secção **Unicode**, falámos de code units já que elas são extremamente importantes para entender quando o caracter codificado em UTF-8 é válido ou não.

Um *Byte Array* vai ser um array com todos os code units de uma string. Esse array é **imutável**, ou seja, só permite que nós vejamos os seus elementos, sem chance de os alterar.


Para criar-mos uma string como uma byte array, fazemos uso de `b""`.

Deixo aqui as regras para Byte Arrays:
* Os caracteres e escapes ASCII são colocado em apenas 1 byte
* Escapes \x e octal produzem o byte correspondente ao valor de escape
* Sequências de escape Unicode criam sequências de bytes codificadas em UTF-8

In [129]:
bytes = b"Julia\x20é\x20top\x20demais"

19-element Base.CodeUnits{UInt8, String}:
 0x4a
 0x75
 0x6c
 0x69
 0x61
 0x20
 0xc3
 0xa9
 0x20
 0x74
 0x6f
 0x70
 0x20
 0x64
 0x65
 0x6d
 0x61
 0x69
 0x73

Podemos transformar este Array de CodeUnits do tipo `UInt8` em um Vector mutável. 

In [130]:
# Podemos ver o code unit no índice 2
bytes[2]

0x75

In [131]:
# Porém se tentarmos modificar
bytes[2] = 0x20

LoadError: setindex! not defined for Base.CodeUnits{UInt8, String}

In [132]:
# Se transformarmos em um Vector de UInt8
transformed_bytes = Vector{UInt8}(bytes)

19-element Vector{UInt8}:
 0x4a
 0x75
 0x6c
 0x69
 0x61
 0x20
 0xc3
 0xa9
 0x20
 0x74
 0x6f
 0x70
 0x20
 0x64
 0x65
 0x6d
 0x61
 0x69
 0x73

In [133]:
# Agora já podemos alterar
transformed_bytes[2] = 0x20

0x20

In [134]:
transformed_bytes

19-element Vector{UInt8}:
 0x4a
 0x20
 0x6c
 0x69
 0x61
 0x20
 0xc3
 0xa9
 0x20
 0x74
 0x6f
 0x70
 0x20
 0x64
 0x65
 0x6d
 0x61
 0x69
 0x73

***Nota***: &#8595;

<hr>

`\xff` e `\uff` são diferentes, o primeiro é a codificação do byte 255 (isto na representação hexadecimal), já o segundo é um code point que é codificado em UTF-8 com 2 bytes. 

<hr>

### Número de versões

Versões em Julia têm o seu próprio "tipo" de String, podendo assim serem definidas por `v""`. Elas são criadas com base nas especificações de [semântica de versões](https://semver.org/), onde  temos:

[1] Versão principal

[2] Versão secundária

[3] Versão do patch

[4] Pre-Release

[5] E por fim build


**Apenas a versão principal é obrigatória**


In [135]:
# Versão do kernel Julia
VERSION

v"1.6.1"

In [136]:
# Versão principal
v"0"

# Versão principal e secundária
v"0.2"

# Versão principal, secundária e patch
v"0.2.1"

# Quando as versões secundárias e de patch são zero é como se não contassem
v"1" == v"1.0" == v"1.0.0"

true

In [137]:
v"0.3-" < v"0.3.2"

true

***Nota:*** &#8595;

<hr>

O `-` significa qualquer versão menor dentro da 0.3

<hr>

In [138]:
v"0.3.1+" > v"0.3.1"

true

***Nota:*** &#8595;
<hr>

O `+` significa qualquer versão maior dentro da 0.3.1

<hr>

### Raw Strings

Strings "cruas" são aquelas que não têm qualquer interpolação ou escape, e podemos defini-las com `raw""`.


Há exceções na parte de escape, pois as aspas `"` precisam de um escape, logo `raw"\"" == "\""`. 

As barras invertidas `\` precisam de escape caso apareçam antes de uma aspa `"`.

Examinemos o exemplo do manual:

In [139]:
println(raw"\\ \\\"")

\\ \"


Os 2 primeiros `\` não levam escape, pois não precedem uma aspa.

No fim da string temos um `\"` responsável pelo escape da aspa, que faz com que o `\` anterior, tenha de ter escape, por isso que precedemos ele com outro `\`.